In [1]:
import numpy as np
from gurobipy import *
import pandas as pd

In [3]:
possible_days = list(range(2, 8+2))
print(possible_days)

[2, 3, 4, 5, 6, 7, 8, 9]


In [2]:
file = "mini_instance.xlsx"
chantiers = pd.read_excel(file, sheet_name = "Chantiers", index_col = "Chantier")
machines = pd.read_excel(file, sheet_name="Machines", index_col="Machine")
sillons_arrivee = pd.read_excel(file, sheet_name="Sillons arrivee", index_col="n°TRAIN")
sillons_depart = pd.read_excel(file, sheet_name="Sillons depart", index_col="n°TRAIN")
correspondances = pd.read_excel(file, sheet_name="Correspondances", index_col="Id wagon")
taches_humaines = pd.read_excel(file, sheet_name="Taches humaines", index_col="Type de train")
roulements_agents = pd.read_excel(file, sheet_name="Roulements agents", index_col="Roulement")

print(correspondances)

FileNotFoundError: [Errno 2] No such file or directory: 'mini_instance.xlsx'

In [6]:
no_mach = machines.shape[0]
no_wag = correspondances.shape[0]
no_mins = 24*12

In [36]:
temps_machines = machines['Duree '].to_numpy() #temps pour chaque machine [0] - DEB,[1] - FOR ,[2] - DEG
indispo_machines = machines['Indisponibilites'].to_numpy() #temps pour chaque machine [0] - DEB,[1] - FOR ,[2] - DEG
print(indispo_machines)

[0 '(6,13:00-13:00);(7, 13:00-13:00)' 0]


In [40]:
# Initialisation du tableau 3D (jours x minutes x machines) rempli de 0 (ouvert)
planning = np.ones((7, 1440, len(indispo_machines)), dtype=int)

# Fonction pour traiter les fermetures des machines
def marquer_fermetures(planning, indispo_machines):
    for index_machine, data in enumerate(indispo_machines):
        if isinstance(data, str):  # Si c'est une chaîne de texte avec les périodes de fermeture
            entrees = data.split(";")  # Séparer les différentes périodes de fermeture

            for entree in entrees:
                entree = entree.strip("()")  # Supprimer les parenthèses
                try:
                    jour, plage_horaire = entree.split(",")  # Séparer le jour et la plage horaire
                    jour = int(jour) - 1  # Convertir en index basé sur 0 (jour 6 devient index 5)

                    if "-" in plage_horaire:
                        heure_debut, heure_fin = plage_horaire.split("-")  # Séparer les horaires
                        
                        # Convertir les heures en minutes
                        debut_min = int(heure_debut[:2]) * 60 + int(heure_debut[3:])  # HH*60 + MM
                        fin_min = int(heure_fin[:2]) * 60 + int(heure_fin[3:])  # HH*60 + MM

                        # Marquer la période comme fermée (1) pour la machine concernée
                        planning[jour, debut_min:fin_min, index_machine] = 0
                    else:
                        print(f"Invalid time range format: {plage_horaire}")
                except ValueError:
                    continue
        else:
            continue

# Appliquer la fonction pour marquer les fermetures dans le planning
marquer_fermetures(planning, indispo_machines)

# Vérification des résultats : afficher les fermetures pour la machine 1 (index 1)
# Exemple : vérifier la fermeture de la machine 1 samedi (jour 5) de 13:00 à 13:30
print(planning[5, 780:800, 1]) 


[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [ ]:
m = Model("Optimisation des machines")

# Variables de décision
X = m.addMVar(shape = (no_mach,no_wag,no_mins),vtype = GRB.BINARY,name = "on/off")
S = m.addMVar(shape = 3,vtype = GRB.INTEGER,name = "heure de debut")

Set parameter Username
Set parameter LicenseID to value 2617662
Academic license - for non-commercial use only - expires 2026-02-03


In [ ]:
# Contraintes
# machine utilisée qu'une fois chaque minute
m.addConstrs((X.sum(0)[i] == 1 for i in range(no_mach)),name = "une_wagon_par_machine")
#
m.add
 

In [9]:
from datetime import datetime, timedelta, time

In [10]:
def time_to_minutes_2(date_str, time_str, jour1,day_1):
    """Convert the time to minutes"""
    if type(date_str) == str:
        date_obj = datetime.strptime(date_str.strip(), "%d/%m/%Y")
    elif type(date_str) == datetime or type(date_str) == pd._libs.tslibs.timestamps.Timestamp:
        date_obj = date_str
    
    # Get the day of the week (0 = Monday, 6 = Sunday...jusqu'à finir le total des jours)
    day_diff = date_obj- jour1
    number_day = day_diff.days-day_1
    
    if type(time_str) == str:
        time_obj = datetime.strptime(time_str.strip(), "%H:%M")
    elif type(time_str) == time:
        time_obj = time_str
    
    total_minutes = number_day * 24 * 60 + time_obj.hour * 60 + time_obj.minute
    return total_minutes

In [20]:
j1 = datetime.strptime("08/08/2022", "%d/%m/%Y")
time_to_minutes_2("08/08/2022", "18:00", j1, 0)/15

72.0